In [1]:
from terminusdb_client import WOQLQuery as WQ
from terminusdb_client import WOQLDataFrame as WDF
from terminusdb_client import WOQLClient, WOQLView

server_url = "https://127.0.0.1:6363"
user = "admin"
account = "admin"
key = "root"
dbpedia = "dbpedia"
dbid = "seshat_combined"
repository = "local"

client = WOQLClient(server_url)
connection = client.connect(user=user,account=account,key=key,db=dbpedia)

In [28]:
conflicts = WQ().woql_and(
    WQ().triple("v:Battle", "type", "scm:MilitaryConflict"),
    WQ().triple("v:Battle", "http://xmlns.com/foaf/0.1/name", "v:Battle_Name"),
    WQ().triple("v:Battle", "scm:combatant", "v:Combatant"),
    WQ().triple("v:Battle", "scm:date", "v:Date"),
    WQ().re("(....)-..-..", "v:Date", ["v:All", "v:Year"]),
    WQ().cast("v:Year", "xsd:integer", "v:Year_Integer")
)

date_query = WQ().select("v:Polity", "v:Start_Integer", "v:End_Integer").woql_and(
    WQ().triple("v:Polity", "type", WQ().iri("terminusdb:///schema#Polity")),
    WQ().triple("v:Polity", "terminusdb:///schema#start", "v:Start_Range"),
    WQ().cast("v:Start_Range", "xsd:string", "v:Start_String"),
    WQ().cast("v:Start_String", "xsd:integer", "v:Start_Integer"),
    WQ().triple("v:Polity", "terminusdb:///schema#end", "v:End_Range"),
    WQ().cast("v:End_Range", "xsd:string", "v:End_String"),
    WQ().cast("v:End_String", "xsd:integer", "v:End_Integer")
)

name_match = WQ().woql_and(
    WQ().triple("v:Polity", "label", "v:Name"),
    WQ().triple("v:Polity", "terminusdb:///schema#alternative_names", "v:YYY"),
    WQ().triple("v:YYY", "terminusdb:///schema#string", "v:AName"),
    WQ().re("(px)?([a-zA-Z\\- ]+)", "v:Combatant", ["v:Combatant_All", "v:Garbage", "v:Pattern"]),
    WQ().woql_or(
      WQ().like("v:Pattern", "v:Name", "v:Match"),
      WQ().like("v:Pattern", "v:AName", "v:Match")
    ), 
    WQ().greater("v:Match", 0.9)
)

id_convert = WQ().woql_and(
    WQ().cast("v:Battle","xsd:string","v:Battle_String"),
    WQ().re("http://dbpedia.org/resource/(.*)", "v:Battle_String", ["v:Doc_Name_All", "v:Doc_Name"]),
    WQ().concat(["terminusdb:///data/", "v:Doc_Name"], "v:ID_String"), 
    WQ().cast("v:ID_String", "owl:Thing", "v:Conflict_ID")
)

year_convert = WQ().woql_and(
    WQ().cast("v:Year_Integer", "xdd:integerRange", "v:Year_Range")
)

candidate_polities = WQ().woql_and(
    WQ().using(f"{user}/{dbid}/local/branch/conflict").woql_and(
        WQ().triple("v:Polity", "type", WQ().iri("terminusdb:///schema#Polity")),
        date_query
    ),
    conflicts,
    WQ().greater("v:End_Integer", "v:Year_Integer"),
    WQ().greater("v:Year_Integer", "v:Start_Integer"),
    WQ().using(f"{user}/{dbid}/local/branch/conflict").woql_and(
        name_match
    )
)

insertion = WQ().using(f"{user}/{dbid}/local/branch/conflict").woql_and(
    WQ().add_triple("v:Conflict_ID", "type", WQ().iri("terminusdb:///schema#Battle")),
    WQ().add_triple("v:Conflict_ID", "label", "v:Battle_Name"),
    WQ().add_triple("v:Conflict_ID", "terminusdb:///schema#end", "v:Year_Range"),
    WQ().add_triple("v:Conflict_ID", "terminusdb:///schema#start",  "v:Year_Range"),
    WQ().add_triple("v:Conflict_ID", "terminusdb:///schema#belligerent", "v:Polity")
)

#seshat_results = WQ().woql_and(candidate_polities, id_convert, year_convert).execute(client)
seshat_results = WQ().woql_and(candidate_polities, id_convert, year_convert, insertion).execute(client, "Adding DBPedia conflicts")

In [29]:
seshat_results

{'@type': 'api:WoqlResponse',
 'api:status': 'api:success',
 'api:variable_names': ['Year_Integer',
  'Year',
  'YYY',
  'Start_Integer',
  'Polity',
  'Pattern',
  'Name',
  'Match',
  'Garbage',
  'End_Integer',
  'Date',
  'Combatant_All',
  'Combatant',
  'Battle_Name',
  'Battle',
  'All',
  'AName',
  'Battle_String',
  'Doc_Name_All',
  'Doc_Name',
  'ID_String',
  'Conflict_ID',
  'Year_Range'],
 'bindings': [{'AName': {'@type': 'http://www.w3.org/2001/XMLSchema#string',
    '@value': 'Last Afghan Empire'},
   'All': {'@type': 'http://www.w3.org/2001/XMLSchema#string',
    '@value': '1757-11-11'},
   'Battle': 'http://dbpedia.org/resource/Battle_of_Amritsar_(1757)',
   'Battle_Name': {'@language': 'en', '@value': 'Battle of Amritsar'},
   'Battle_String': {'@type': 'http://www.w3.org/2001/XMLSchema#string',
    '@value': 'http://dbpedia.org/resource/Battle_of_Amritsar_(1757)'},
   'Combatant': {'@language': 'en', '@value': 'Durrani Empire'},
   'Combatant_All': {'@type': 'http: